# Proof of concept

In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import uuid
from pandarallel import pandarallel
from tqdm.notebook import tqdm
from copy import deepcopy

pandarallel.initialize()

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
from mc_moea import *

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read position data

In [3]:
data = []
fnames = glob.glob("position/*.dat")
for fname in fnames:
    X = pickle.load(open(fname, "rb"))
    data.append(X)

In [15]:
position_df = data[0][:2000]

In [16]:
position_df.shape

(2000, 2)

### Stations

In [24]:
station_types

,types,frequency,capacity,cost
0,900 MHz Type I,900,800,1150000
1,900 MHz Type II,900,1200,1500000
2,1800 MHz Type I,1800,850,880000
3,1800 MHz Type II,1800,1250,1220000
4,2600 MHz Type I,2600,800,950000
5,2600 MHz Type II,2600,1300,1350000


### Simulation parameters

In [33]:
max_station = 8
n_population = 300
crossover_p = 0.3
mutation_p = 0.1
elitism_ratio = 0.5
trim_policy = "best"  # "random" or "best"
num_iters = 400
checkpoint_every = 1

### Initial population

In [34]:
n_station = 8

In [35]:
# stations_l = [generate_station(n) for n in np.random.randint(1,max_station+1, size=n_population)]
stations_l = [generate_station(n) for n in [n_station]*n_population]

solution_pool = [Solution(stations_l[i]) for i in range(n_population)]

solution_df = pd.DataFrame({"solution_id":[x.id for x in solution_pool],
                            "solution_obj": solution_pool,
                            "from_generation":[-1]*n_population,
                            "from_method":["Randomize"]*n_population
                           })

### Iteration

In [36]:
iteration_best = []
iteration_pattern = []
iteration_df = []

In [37]:
start = 0
num_iters = 100
for i in tqdm(range(start, num_iters)):
    print(f"Iteration {i+1}/{num_iters}")
    
    position = position_df
    
    # print(f"\tFitting with pattern {data_pattern}...")
    print(f"\tComputing fitness value...")
    solution_df = compute(solution_df, position)
    solution_df = solution_df.sort_values(by="fitness_value")
    
    new_solution_df = pd.DataFrame()
    if crossover_p:
        print(f"\tCrossover...")
        crossover_df = crossover(solution_df, crossover_p)
        crossover_df["from_method"] = ["Crossover"]*crossover_df.shape[0]
        new_solution_df = pd.concat([new_solution_df, crossover_df])
    if mutation_p:
        print(f"\tMutation...")
        mutation_df = mutation(solution_df, mutation_p)
        mutation_df["from_method"] = ["Mutation"]*mutation_df.shape[0]
        new_solution_df = pd.concat([new_solution_df, mutation_df])
        
    if new_solution_df.shape[0] != 0:
        print(f"\tComputing fitness value of new solutions...")
        new_solution_df["solution_id"] = new_solution_df["solution_obj"].parallel_apply(lambda x:x.id)
        new_solution_df = compute(new_solution_df, position)
        new_solution_df["from_generation"] = [i]*new_solution_df.shape[0]
    
    if elitism_ratio:
        print(f"\tPerforming elite selection and reinitialize...")
        n_elite = int(n_population * elitism_ratio)
        n_reinit = n_population - n_elite
        solution_df = solution_df.sort_values(by="fitness_value").head(n_elite)
        reinit_stations_l = [generate_station(n) for n in np.random.randint(1, max_station+1, size=n_reinit)]
        # reinit_stations_l = [generate_station(n) for n in [n_station]*n_reinit]
        reinit_pool = [Solution(s) for s in reinit_stations_l]
        reinit_solution_df = pd.DataFrame({"solution_id":[x.id for x in reinit_pool],
                                          "solution_obj": reinit_pool
                                         })
        reinit_solution_df = compute(reinit_solution_df, position)
        reinit_solution_df["from_generation"] = [i]*reinit_solution_df.shape[0]
        reinit_solution_df["from_method"] = ["Randomize"]*reinit_solution_df.shape[0]
        solution_df = pd.concat([solution_df, reinit_solution_df], ignore_index=True)
    
    print(f"\tTrimming result...")
    if trim_policy=="random":
        print("\t\tRandom trim")
        solution_df = pd.concat([solution_df, new_solution_df], ignore_index=True).sample(n=n_population).reset_index(drop=True)
    else:
        print("\t\tRetain best trim")
        solution_df = pd.concat([solution_df, new_solution_df], ignore_index=True).sort_values(by="fitness_value").head(n_population).reset_index(drop=True)
    
    solution_df = solution_df.sort_values(by="fitness_value")
    best = deepcopy(solution_df[solution_df['fitness_value']==min(solution_df['fitness_value'])])
    iteration_best.append(best)
    # iteration_pattern.append(data_pattern)
    iteration_df.append(deepcopy(solution_df[:10]))
    print(f"\tBest fitness of this iteration: {best['fitness_value'].iloc[0]}")
    if i%checkpoint_every == 0:
        fname = f"solutions/poc/solution-{i:04d}.pkl.bz2"
        print(f"\tSaving checkpint at {fname}")
        solution_df.to_pickle(fname, compression="bz2")
    print(f"\tDone!")

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 1/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Performing elite selection and reinitialize...
	Trimming result...
		Retain best trim
	Best fitness of this iteration: 0.996123567123411
	Saving checkpint at solutions/poc/solution-0000.pkl.bz2
	Done!
Iteration 2/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Performing elite selection and reinitialize...
	Trimming result...
		Retain best trim
	Best fitness of this iteration: 0.9725110893327982
	Saving checkpint at solutions/poc/solution-0001.pkl.bz2
	Done!
Iteration 3/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Performing elite selection and reinitialize...
	Trimming result...
		Retain best trim
	Best fitness of this iteration: 0.9725110893327982
	Saving checkpint at solutions/poc/solution-0002.pkl.bz2
	Done!
Iteration 4/100
	Computing fitness value...
